In [1]:
import mainCode as Haha
import tensorflow as tf

# basic data

In [2]:
odlist_4 = [
    'fruity', 'sweet', 'green', 'floral', 'woody', 'herbaceous', 'fatty'
]

getdata = Haha.BasicData(which_data='mine')
getdata.readOD(50, od_selected=odlist_4)
smiles_list = getdata.readSmiles()
print(len(smiles_list))

getdata.which_data = 'odorless'
smiles_list_odorless = getdata.readSmiles(filelist=['summary_odorless.txt'])
print(len(smiles_list_odorless))

smiles_list.extend(smiles_list_odorless)
print(len(smiles_list))

number of all type odor descriptors = 970
appear times >= 50 odor descriptors have 71
they are
['fruity', 'sweet', 'green', 'floral', 'woody', 'herbaceous', 'fatty', 'fresh', 'waxy', 'rose', 'spicy', 'citrus', 'earthy', 'tropical', 'oily', 'nutty', 'roasted', 'sulfurous', 'balsam', 'apple', 'dry', 'powdery', 'phenolic', 'vegetable', 'pineapple', 'bland', 'aldehydic', 'musty', 'creamy', 'minty', 'meaty', 'metallic', 'honey', 'onion', 'berry', 'balsamic', 'jasmin', 'spice', 'banana', 'pungent', 'animal', 'amber', 'musk', 'caramel', 'pear', 'burnt', 'camphor', 'coconut', 'orange', 'coffee', 'mushroom', 'garlic', 'melon', 'natural', 'violet', 'leafy', 'peach', 'tobacco', 'pine', 'geranium', 'vanilla', 'cocoa', 'sulfury', 'nut', 'buttery', 'cooked', 'lily', 'hyacinth', 'apricot', 'cherry', 'almond']
final od_times with all samples: {'fruity': 1209, 'sweet': 1059, 'green': 1034, 'floral': 851, 'woody': 598, 'herbaceous': 532, 'fatty': 404}
3708
879
4587


In [3]:
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

# ODs prediction

In [4]:
test = Haha.Transformer2OD_tada()
test.molData_atomicF(smiles_list, dist_value='exp', atomH=False, failed_mol=None, rseed=1)
test.odData(getdata.od_mat_ori, getdata.od_name)

Transformer2OD_tada
transformer2od
54 mol cannot compute distance, they are: [1992, 1994, 1999, 2005, 2012, 2016, 2018, 2024, 2513, 2565, 3057, 3068, 3260, 3261, 3270, 3289, 3329, 3553, 3554, 3650, 3833, 3871, 3883, 3889, 3938, 4179, 4189, 4235, 4335, 4336, 4337, 4409, 4442, 4469, 4475, 4491, 4502, 4522, 4523, 4524, 4526, 4539, 4543, 4560, 4561, 4565, 4568, 4569, 4574, 4578, 4580, 4582, 4583, 4586]
num_sample : 4533
mask_mat: (4533, 1, 1, 60)
mol_atom_feat.shape: (4533, 60, 71)
distance value type is: exp
all_adj.shape = : (4533, 60, 60)
all_dist.shape = : (4533, 60, 60)
od_shuffled.shape =  (4533, 7)
positive sample distribution in test set:
{'fruity': 201.0, 'sweet': 144.0, 'green': 150.0, 'floral': 125.0, 'woody': 89.0, 'herbaceous': 93.0, 'fatty': 77.0}
positive sample distribution in test set:
{'fruity': 1007.0, 'sweet': 913.0, 'green': 880.0, 'floral': 726.0, 'woody': 504.0, 'herbaceous': 437.0, 'fatty': 324.0}


In [5]:
test.use_adj_dist = 'both'
hpara = {
            'num_heads': 6,
            'single_attn_dim': 30,
            'num_encoderLayer': 2,
            'num_decoderLayer': 3, 
            'learning_rate': 0.00002, 
            'temperature': 0.7
        }

record_dir='/tf/haha/code/logs/0607/od_pred'

weight_path = '/tf/haha/save_weight/0607/od_pred/'


In [6]:
test.modelTrain2(hpara, epochs=80, batch_size=32, record_dir=record_dir, save_threshold=0.54, save_path=weight_path, normal_init=False)

/tf/haha/code/logs/0607/od_pred
modelBuild2-temperature 0.700000
(3775, 1, 1)
(3775, 60, 71)
(3775, 60, 60)
(3775, 60, 60)
(3775, 1, 1, 60)
(3775, 7)
0 WARNING:tensorflow:Model was constructed with shape (32, 1, 1) for input KerasTensor(type_spec=TensorSpec(shape=(32, 1, 1), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'"), but it was called on an input with incompatible shape (31, 1, 1).
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 weight保存开始：Tue Jun  7 06:01:50 2022 weight保存完成 Tue Jun  7 06:01:50 2022
36 37 38 39 40 weight保存开始：Tue Jun  7 06:02:22 2022 weight保存完成 Tue Jun  7 06:02:22 2022
41 42 43 44 45 weight保存开始：Tue Jun  7 06:02:53 2022 weight保存完成 Tue Jun  7 06:02:53 2022
46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 weight保存开始：Tue Jun  7 06:06:07 2022 weight保存完成 Tue Jun  7 06:06:08 2022
77 78 79 {'avg': 0.5505764284063072, 'avg_pcs': 0.4970613

(<keras.engine.functional.Functional at 0x7f5c4c4f4ca0>,
 {'avg': 0.5505764284063072,
  'avg_pcs': 0.4970613845757076,
  'avg_rc': 0.6230546874659402,
  'fruity': 0.6744186095607286,
  'sweet': 0.5254691861174295,
  'green': 0.5207756238779121,
  'floral': 0.5810810819695447,
  'woody': 0.6210526433578818,
  'herbaceous': 0.43434344006307196,
  'fatty': 0.49689441389758104})

# substructure prediction

In [ ]:
sub_dict = {
    '0': 'cCc',
    '1': 'CC(C)(C)C',
    '2': 'CC(C)C', # 三叉的碳
    '3': 'C1=CCCC1', # 五元环带个双键
    '4': 'C12CC(CC1)CC2',
    '5': '[CD4]',
    '6': '[r5]',
    '7': '[r4]',
    '8': 'C1CCC1',
    '9': 'C1CCCC1',
    '10': 'C(=O)O&&CC(C)C',
    '11': 'C(=O)O&&[r5]',
    '12': 'C=C-C=C&&CC(C)C',
    '13': 'SS',
    '14': '[r6]&&[r5]',
}

od_name = []
sub_list = []
for sub in sub_dict:
    od_name.append(sub)
    sub_list.append(sub_dict[sub])

otherpairs = [[12,14]]
od_name.append('12or14')

In [ ]:
test = Haha.Transformer2Sub()
test.molData_atomicF(smiles_list, dist_value='exp', atomH=False, failed_mol=None, rseed=0)
test.odData(smiles_list, sub_list=sub_list, od_name=od_name, otherpairs=otherpairs)

In [ ]:
hpara = {
            'num_heads': 6,
            'single_attn_dim': 30,
            'num_encoderLayer': 2,
            'num_decoderLayer': 4, 
            'learning_rate': 0.00003, 
            'temperature': 0.7
        }

record_dir='/tf/haha/code/logs/0607/sub_pred'

weight_path = '/tf/haha/save_weight/0607/sub_pred/'

In [ ]:
test.modelTrain2(hpara, epochs=200, batch_size=32, record_dir=record_dir, save_threshold=0.54, save_path=weight_path, normal_init=False)

# draw attention

In [ ]:
# make output
out_train = []
out_test = []
for i in range(test.feature_od):
    out_train.append(test.od_train[:, i])
    out_test.append(test.od_test[:, i])
out_train = tuple(out_train)
out_test = tuple(out_test)

In [ ]:
test.use_adj_dist = 'both'
model = test.modelBuild2(batch_size=32, num_heads=6, single_attn_dim=30, feedforward_dim=180, num_encoderLayer=2, 
                    num_decoderLayer=3, dropout_rate=0.1, lr=0.00002, compile=True, normal_init=False, temperature=0.7)

model.load_weights('/tf/haha/save_weight/0607/od_pred/')

if len(test.test_input) == test.input_items-1:
    test.test_input = list(test.test_input)
    test.test_input.append(test.od_test)
    test.test_input = tuple(test.test_input)

In [ ]:
pred = model(test.test_input)
eva_dict = model.evaluate(test.test_input, out_test, return_dict=True, batch_size=2)
print(test.calFscore(eva_dict))

In [ ]:
# attnDraw
attn = test.attnExtract(model, num_cls=test.feature_od, trainORtest='test', dc_layer=True)
attn = [attn[-1]] # if draw each dc layer, delete this line
pred = model(test.test_input)
savepath = '/tf/haha/img_result/0607/6_30_23/'
test.drawAttn(attn, smiles_list, pred, savepath, max_draw=500, per_head=False, trainORtest='test', attn_c=0.7, posneg='positive')